In [2]:
import torch
import random
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
import  pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
# Directly Downloading dataset from kaggle and loading
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("tarkkaanko/amazon")

print("Path to dataset files:", os.listdir(path))

Path to dataset files: ['amazon_reviews.csv']


# 1. Data Preprocessing

In [5]:
df = pd.read_csv(f'{path}/amazon_reviews.csv')
print(df.shape)
df.head(5)

(4915, 12)


,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [6]:
# Removing the other nucessary columns
df = df[['reviewText','overall']]

# Convert ratings to binary labels (-1=negative, 0=neutral, 1=positive)
df['label'] = df['overall'].apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2 if x>3 else None)

<ipython-input-6-2fa8e71d1806>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['overall'].apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2 if x>3 else None)


In [7]:
df.head()

,reviewText,overall,label
0,No issues.,4.0,2
1,"Purchased this for my device, it worked as adv...",5.0,2
2,it works as expected. I should have sprung for...,4.0,2
3,This think has worked out great.Had a diff. br...,5.0,2
4,"Bought it with Retail Packaging, arrived legit...",5.0,2


In [8]:
df['label'].value_counts()

,count
label,
2,4449
0,324
1,142


In [9]:
def clean_text(text):
    # Checks if the text are actully string or not
    if not isinstance(text, str):
        text = str(text)
    # Convet to lowerCase
    text = text.lower()
    # Remove Urls
    #text = re.sub('https?://\S+|www\.\S+','',text)
    # Remove Html tags
    #text = re.sub(r'<.*?>','',text)
    # Remove special characters ann numbers
    text = re.sub(r'[^a-zA-Z\s]','',text)

    # Split the sentences into words
    text= text.split()

    # Applying stopwords and lemmatization
    text = [lemmatizer.lemmatize(word) for word in text if not word in stopwords.words('english')]

    # Remove extra whitespace
    text = ' '.join(text)
    return text

df['cleaned_reviewText'] = df['reviewText'].apply(clean_text)

In [10]:
df.head()

,reviewText,overall,label,cleaned_reviewText
0,No issues.,4.0,2,issue
1,"Purchased this for my device, it worked as adv...",5.0,2,purchased device worked advertised never much ...
2,it works as expected. I should have sprung for...,4.0,2,work expected sprung higher capacity think mad...
3,This think has worked out great.Had a diff. br...,5.0,2,think worked greathad diff bran gb card went s...
4,"Bought it with Retail Packaging, arrived legit...",5.0,2,bought retail packaging arrived legit orange e...


In [11]:
df.isnull().sum()

,0
reviewText,1
overall,0
label,0
cleaned_reviewText,0


In [12]:
df.dropna(inplace = True)

# 2. Tokenization (No Manual Vectorization Needed for Transformers)

#### Transformers handle vectorization internally through their tokenizers:

In [13]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['cleaned_reviewText'],padding='max_length',truncation= True)

# Filling the nan valuer with empth string
df['cleaned_reviewText'] = df['cleaned_reviewText'].fillna('').astype(str)

# Converting to HuggingFace Dataset format
from datasets import Dataset
dataset = Dataset.from_pandas(df)
tokenize_dataset = dataset.map(tokenize_function,batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4914 [00:00<?, ? examples/s]

In [15]:
print(tokenize_dataset)

Dataset({
    features: ['reviewText', 'overall', 'label', 'cleaned_reviewText', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 4914
})


In [16]:
print(tokenize_dataset.features)

{'reviewText': Value(dtype='string', id=None), 'overall': Value(dtype='float64', id=None), 'label': Value(dtype='int64', id=None), 'cleaned_reviewText': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


# 3. Model Traning using Transformers Models

In [17]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments , Trainer
import numpy as np
import evaluate

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels= 5 # hyperparamerter
)

#split dataset
train_test = tokenize_dataset.train_test_split(test_size=0.2)

# Metrics
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = np.argmax(logits,axis = -1)
    return metric.compute(predictions=predictions,references = labels)

# Training arguments
training_args = TrainingArguments(
    output_dir = './results',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none' #disable wandb/etc for simplicty

)
# Treiner
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=train_test['train'],
    eval_dataset=train_test['test'],
    compute_metrics= compute_metrics
)

# Train
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


In [21]:
from transformers import pipeline
import torch

def predict_sentiment(reviews, model_path='./results/checkpoint-738', batch_size=16):

    # Load trained model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

    # Create pipeline
    classifier = pipeline(
        "text-classification",
        model=model,
        tokenizer=tokenizer,
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
        batch_size=batch_size
    )

    # Single vs batch processing
    is_single = isinstance(reviews, str)
    inputs = [reviews] if is_single else reviews

    # Get predictions
    raw_preds = classifier(inputs)

    # Format results
    def format_pred(pred):
        star = int(pred['label'].split('_')[-1])  # Convert LABEL_1 → 1
        return {
            'stars': star,
            'sentiment': 'positive' if star > 3 else 'negative' if star < 3 else 'neutral',
            'confidence': float(pred['score'])
        }

    results = [format_pred(p) for p in raw_preds]
    return results[0] if is_single else results

In [25]:
# Example usage
test_reviews = [
    "This product changed my life! Worth every penny.",
    "Terrible quality. Broke after 2 days.",
    "It's okay, but the packaging was damaged.",
    "The item works as described, nothing special.",
    'good porduct',
    'no issue',
    'love it ',
    'amazing'
]

predictions = predict_sentiment(test_reviews)

for review, pred in zip(test_reviews, predictions):
    print(f"Review: {review[:50]}...")
    print(f"Predicted: {pred['stars']} stars ({pred['sentiment']}, {pred['confidence']:.2f} confidence)")
    print("-" * 80)

Device set to use cuda


Review: This product changed my life! Worth every penny....
Predicted: 2 stars (negative, 0.99 confidence)
--------------------------------------------------------------------------------
Review: Terrible quality. Broke after 2 days....
Predicted: 0 stars (negative, 0.49 confidence)
--------------------------------------------------------------------------------
Review: It's okay, but the packaging was damaged....
Predicted: 2 stars (negative, 0.89 confidence)
--------------------------------------------------------------------------------
Review: The item works as described, nothing special....
Predicted: 2 stars (negative, 0.97 confidence)
--------------------------------------------------------------------------------
Review: good porduct...
Predicted: 2 stars (negative, 0.99 confidence)
--------------------------------------------------------------------------------
Review: no issue...
Predicted: 2 stars (negative, 0.96 confidence)
--------------------------------------------------